# RAG 구현하기

Retriever (검색, 회수)
-> Augmented (증강, 기존에 있는 데이터를 늘린다)
-> Generation (생성, LLM이 답변을 생성함)

### 형태소 끊기 -> 단어 정제 -> 기호 제거, 단어 카운팅, 딕셔너리
문서 가져오기 -> (자연어 처리) -> 임베딩(글자->숫자) -> 벡터DB(벡터화/'방향성' 글자의 유사도를 측정 가능)

=> 검색/증강/생성

In [2]:
# llm 관련 라이브러리
!pip install langchain openai langchain-community

# 토큰화를 지원하는 라이브러리
!pip install tiktoken

# 벡터 간의 유사도 검색 알고리즘
!pip install rank_bm25

# 벡터 DB 라이브러리
!pip install chromadb faiss-cpu

# PDF 의 데이터를 추출하는 라이브러리
!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 45.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 83.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 74.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 96.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 8.2 MB/s eta 0:00:

In [32]:
# openai, 토큰화(tiktoken)
import openai, tiktoken

# 임베딩 지원하는 라이브러리(임베딩 규칙을 가져다 씀)
# from langchain.embeddings import HuggingFaceEmbeddings
from langchain.embeddings import HuggingFaceBgeEmbeddings
# 벡터 DB
from langchain import FAISS

# PDF로 된 자료를 읽고, 데이터를 추출하기 위함
from langchain.document_loaders import PyPDFLoader
from PyPDF2 import PdfReader
from langchain.text_splitter import CharacterTextSplitter

# 질의응답(생성 담당)
from langchain.chains.question_answering import load_qa_chain

In [4]:
import os, openai
from google.colab import userdata

os.environ['OPENAI_API_KEY'] = userdata.get('sesac7_openAI_key')

# RAG를 안 쓴 상태에서 질의응답.

In [7]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

# 기본 llm을 활용해서 질문 창을 염
chat = ChatOpenAI()

# {대상}에 대한 질의응답 템플릿 정의
template_message = ChatPromptTemplate.from_template('{input}이 뭐야?')
# {대상} == 프롬프트 엔지니어링
message = template_message.format_messages(input='프롬프트 엔지니어링')

response = chat(message)
print(response.content)

프롬프트 엔지니어링은 컴퓨터 시스템에서 사용자와 상호작용하기 위해 사용되는 명령 프롬프트(Command Prompt)나 터미널(Terminal) 등을 통해 시스템과 통신하는 기술이다. 이를 통해 사용자는 명령어를 입력하여 시스템을 제어하거나 작업을 수행할 수 있다. 프롬프트 엔지니어링은 주로 시스템 관리, 자동화, 스크립팅 등에 활용되며, IT 전문가나 시스템 관리자들이 네트워크, 서버, 데이터베이스 등을 관리하는 데에 사용된다.


## RAG를 통해 llm이라는 도메인에 충실한 답변을 끌어내자.

1. 자료를 가져옴(pdf, web, text, csv...)
2. 자료를 토큰으로 만듦 / load_and_split => '청크'
3. 청크화된 문서를 임베딩 -> 벡터 DB에 집어넣음
4. 검색 from 벡터DB (리트리버)
    
    " 잘라둔 텍스트 덩어리를 리트리버 알고리즘에 던져서, '질문과 가장 유사한' 텍스트 덩어리를 고름"

5. 4의 결과로 텍스트 '증강'
6. 생성(GPT에 5의 내용을 전달, 답변을 만들어 냄)

In [8]:
pdf_path = '/content/프롬프트엔지니어링.pdf'

pdf_content = PdfReader(pdf_path)

In [11]:
pdf_content.pages[0]

{'/Type': '/Page',
 '/Parent': {'/Type': '/Pages',
  '/Count': 22,
  '/Kids': [IndirectObject(3, 0, 137235726619408),
   IndirectObject(17, 0, 137235726619408),
   IndirectObject(21, 0, 137235726619408),
   IndirectObject(26, 0, 137235726619408),
   IndirectObject(32, 0, 137235726619408),
   IndirectObject(34, 0, 137235726619408),
   IndirectObject(38, 0, 137235726619408),
   IndirectObject(40, 0, 137235726619408),
   IndirectObject(44, 0, 137235726619408),
   IndirectObject(46, 0, 137235726619408),
   IndirectObject(48, 0, 137235726619408),
   IndirectObject(52, 0, 137235726619408),
   IndirectObject(55, 0, 137235726619408),
   IndirectObject(62, 0, 137235726619408),
   IndirectObject(64, 0, 137235726619408),
   IndirectObject(67, 0, 137235726619408),
   IndirectObject(77, 0, 137235726619408),
   IndirectObject(80, 0, 137235726619408),
   IndirectObject(82, 0, 137235726619408),
   IndirectObject(86, 0, 137235726619408),
   IndirectObject(90, 0, 137235726619408),
   IndirectObject(99, 

In [12]:
texts = ''

for page in pdf_content.pages:
    texts += page.extract_text()

In [13]:
texts

'프롬프트 엔지니어링 :\n업무능력의 혁신INDEX\n1프롬프트 엔지니어링이란 무엇인가 ?\n2프롬프트 엔지니어링의 구성요소\n3프롬프트 엔지니어링 스킬\n4프롬프트 엔지니어링 실습Generative AI?프롬프트 엔지니어링이란 ?\ndesigning affective inputs and fine-tuing parameters\nforaimodels togetthedesired results\n프롬프트 엔지니어링이란 ?\n①원하는결과물을 얻기위해\n②인풋,파라미터 ,아웃풋을\n③명확하게 ,정확하게 ,창의적으로\n④생성형 ai에게제시하는 것\n어학스킬+문제해결능력 +비판적사고프롬프트 엔지니어링의 구성요소\n1인풋(inputs)\n출력물의 기반이되는고유한데이터와 정보 .\n2파라미터 (parameter)\n작업지시 및맥락정보등\n사용자의 요청과명령을수행하는 데필요한행동 (작업지시 ),작업지시를\n이해하기 위한배경정보 (맥락 )등을의미\n3아웃풋 (outputs)\n결과물의 유형 ,형식 ,구조에대한제한과기대값 .\n원하는출력물의 형태를구체적으로 가이드해야 한다 .프롬프트 엔지니어링의 스킬\n1명령(task)\n2맥락(context)\n3페르소나 (persona)4예시(example)\n5포맷(format)\n6어조(tone)프롬프트 엔지니어링의 스킬\n1명령(task)1프롬프트 ,1명령\n서술어로 ,간결히\n~를제시해줘 .\n~를제안해줘 .프롬프트 엔지니어링의 스킬\n2맥락(context)현재어떤상황인지 ?\n의도와목표가무엇인지 ?\n우려되는 사항은무엇인지 ?\n답변에있어고려해야 할제약 /규칙은무엇인지 ?프롬프트 엔지니어링의 스킬\n3페르소나 (persona)롤플레이처럼 지시하기\n생성형 AI에게직무와역할을부여\n구체적인 전문가의 명칭을활용할것\n전문용어를포함할수록더전문적인 산출물을 냄프롬프트 엔지니어링의 스킬\n4예시(example)Few shot\n프롬프트 엔지니어링의 스킬\n5포맷(format)결과물의 형식및분량을구체적으로 지정\n결과물의 내용구성이나 아웃라인

# 2. 자료를 토큰으로 만듦(청크)

In [24]:
def preprocess_text(text):
    # 청크로 자르기
    # chunk_size = 길이를 얼마로 자를래?
    # chunk_overlap
    # 나는 코딩을 좋아해요.
    # 나는 코딩을
    #             좋아해요.
    # overlap이 0
    # 나는 코딩을 좋
    #        딩을 좋아해요.
    # overlap이 있다.
    splitter = CharacterTextSplitter(chunk_size=500,
                          chunk_overlap=50,
                          length_function=len,
                          separator='\n')

    chunk = splitter.split_text(text)

    # 청크화된 문서를 임베딩 -> 벡터 DB에 집어넣음
    embeddings = HuggingFaceBgeEmbeddings(model_name='BAAI/bge-m3')

    # 임베딩을 실행, 바로 벡터 DB에 인풋
    documents = FAISS.from_texts(chunk, embeddings)
    return documents

In [15]:
docs = preprocess_text(texts)

/tmp/ipython-input-4151375829.py:20: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name='BAAI/bge-m3')
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [26]:
docs

In [27]:
doc = docs.similarity_search("프롬프트 엔지니어링이 뭐야?")
print(doc)

[Document(id='13381312-e12f-45f4-9a2f-d37b72e6117e', metadata={}, page_content='프롬프트 엔지니어링 :\n업무능력의 혁신INDEX\n1프롬프트 엔지니어링이란 무엇인가 ?\n2프롬프트 엔지니어링의 구성요소\n3프롬프트 엔지니어링 스킬\n4프롬프트 엔지니어링 실습Generative AI?프롬프트 엔지니어링이란 ?\ndesigning affective inputs and fine-tuing parameters\nforaimodels togetthedesired results\n프롬프트 엔지니어링이란 ?\n①원하는결과물을 얻기위해\n②인풋,파라미터 ,아웃풋을\n③명확하게 ,정확하게 ,창의적으로\n④생성형 ai에게제시하는 것\n어학스킬+문제해결능력 +비판적사고프롬프트 엔지니어링의 구성요소\n1인풋(inputs)\n출력물의 기반이되는고유한데이터와 정보 .\n2파라미터 (parameter)\n작업지시 및맥락정보등\n사용자의 요청과명령을수행하는 데필요한행동 (작업지시 ),작업지시를\n이해하기 위한배경정보 (맥락 )등을의미\n3아웃풋 (outputs)\n결과물의 유형 ,형식 ,구조에대한제한과기대값 .'), Document(id='da625baf-e8ce-45b0-a1d5-f8e25b38023b', metadata={}, page_content='핵심요구사항 도출\n주제에대한핵심요구사항을 도출한다 .\n“그려줘”\n그림에대한상세묘사\n예시제안 (fewshot)\n영어로\n?프롬프트 엔지니어링은\n업무생산성과 창의성을 높일수\n있는혁신적인 기술\n다양한분야에서 활용가능한\n실용적인 스킬셋\n윤리성과 품질관리에대한주\n의가필요\n결론및질의응답\n혁신적 실용적 주의!References\nPrompt Engineering For Everyone with ChatGPT and GPT -4,Andrei Gheorghiu\nPrompt Engineering for Generative AI ,James Phoenix, Mike T

In [28]:
llm = ChatOpenAI(temperature=0)

# 정보를 증강 생성 시킬 때, 옵션을 배정
chain = load_qa_chain(llm=llm,
                      chain_type= 'stuff'    # 정보를 어떻게 인식할 것인가? stuff:한번에 인식
                      )

response = chain.run(input_documents=doc,
          question='프롬프트 엔지니어링이 뭐야?')
print(response)

프롬프트 엔지니어링은 원하는 결과물을 얻기 위해 인풋, 파라미터, 아웃풋을 명확하고 정확하게, 창의적으로 생성형 AI에게 제시하는 것을 말합니다. 이는 업무생산성과 창의성을 높일 수 있는 혁신적인 기술이며, 다양한 분야에서 활용 가능한 실용적인 스킬셋입니다.


In [29]:
# 벡터 DB 저장
# 벡터 DB 한 번 만들고 재활용
docs.save_local('/content/faiss')

In [33]:
# 저장할 때 썼던 동일 모델로 임베딩 초기화
embeddings = HuggingFaceBgeEmbeddings(model_name='BAAI/bge-m3')

# 로컬에 저장된 DB 불러오기
vectorstore = FAISS.load_local(
    "/content/faiss",
    embeddings,
    allow_dangerous_deserialization=True  # colab/jupyter에서는 보통 이 옵션 필요
)
# 검색 테스트
result = vectorstore.similarity_search("프롬프트 엔지니어링에서 쓸 수 있는 스킬들 뭐 없어?", k=2)
print(result)

/tmp/ipython-input-941541117.py:2: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceBgeEmbeddings(model_name='BAAI/bge-m3')


[Document(id='6fd66beb-4d03-4592-889f-356e80b87835', metadata={}, page_content='프롬프트 엔지니어링의 스킬\n5포맷(format)결과물의 형식및분량을구체적으로 지정\n결과물의 내용구성이나 아웃라인을 제공\nhttps://sabarada.tistory.com /209프롬프트 엔지니어링의 스킬\n6어조(tone)가상의청중을제시(‘5살인것처럼설명해줘’)\n‘상냥하게 ’,‘친절하게 ’\n필요하다면 예시를제공후,예시의잘된부분을설명하여 가이드라인을 명확히프롬프트 엔지니어링의 스킬\n그외의팁\n1.영어를활용할것\n2.강조표시를활용할것\n3.결과물에 대해피드백을 제공하며 ,답변을순차적으로 이끌어낼것\n4.생성형 AI를협박할것.(보상을제시할것)프롬프트 엔지니어링 예제\n주제선정\n프롬프트 엔지니어링을 적용할업무주제를정한다 .\n프롬프트 엔지니어링 실습예제\n핵심요구사항 도출\n주제에대한핵심요구사항을 도출한다 .\n최적프롬프트 작성\n요구사항을 반영하여 프롬프트를 작성한다 .주제선정\n프롬프트 엔지니어링을 적용할업무주제를정한다 .\n핵심요구사항 도출\n주제에대한핵심요구사항을 도출한다 .\n“그려줘”\n그림에대한상세묘사'), Document(id='13381312-e12f-45f4-9a2f-d37b72e6117e', metadata={}, page_content='프롬프트 엔지니어링 :\n업무능력의 혁신INDEX\n1프롬프트 엔지니어링이란 무엇인가 ?\n2프롬프트 엔지니어링의 구성요소\n3프롬프트 엔지니어링 스킬\n4프롬프트 엔지니어링 실습Generative AI?프롬프트 엔지니어링이란 ?\ndesigning affective inputs and fine-tuing parameters\nforaimodels togetthedesired results\n프롬프트 엔지니어링이란 ?\n①원하는결과물을 얻기위해\n②인풋,파라미터 ,아웃풋을\n③명확하게 ,정확하게 ,창의적으로\n④생성형 ai에게제시하는 것\n어학스킬+문제해결능